In [1]:
import pandas as pd
import psycopg2
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report


In [2]:
# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname="SQLTEST", 
    user="postgres", 
    password="Admin", 
    host="localhost", 
    port="5432"
)

In [3]:
# SQL query to extract the data
query = """
WITH last_order_date AS (
    SELECT 
        MAX(created_at) AS max_date
    FROM 
        public.orders
)
SELECT 
    p.id AS product_id,
    pn.name AS product_name,
    c.name AS category_name,
    pvs.stock AS current_stock,
    pvp.price AS current_price,
    COALESCE(SUM(o.total_amount), 0) AS total_sales_last_3_months,
    COALESCE(AVG(pr.rating), 0) AS average_rating,
    COUNT(DISTINCT o.id) AS total_orders_last_3_months,
    (SELECT COUNT(*) 
     FROM public.orders o2 
     WHERE o2.created_at >= (SELECT max_date FROM last_order_date) - INTERVAL '6 months' 
     AND o2.id IN (SELECT pr2.order_id FROM public.product_ratings pr2 WHERE pr2.product_id = p.id)) AS total_orders_last_6_months,
    (SELECT COUNT(*) 
     FROM public.orders o3 
     WHERE o3.created_at >= (SELECT max_date FROM last_order_date) - INTERVAL '3 months' 
     AND o3.id IN (SELECT pr3.order_id FROM public.product_ratings pr3 WHERE pr3.product_id = p.id)) AS total_orders_last_3_months,
    pv.weight AS product_weight,
    pv.status AS variation_status
FROM 
    public.products p
JOIN 
    public.product_names pn ON p.name_id = pn.id
JOIN 
    public.categories c ON pn.category_id = c.id
JOIN 
    public.product_variations pv ON p.id = pv.product_id
JOIN 
    public.product_variation_stocks pvs ON pv.id = pvs.product_variation_id
LEFT JOIN 
    public.product_variation_prices pvp ON pv.id = pvp.product_variation_id
LEFT JOIN 
    public.product_ratings pr ON pr.product_id = p.id
LEFT JOIN 
    public.orders o ON o.id = pr.order_id AND o.created_at >= (SELECT max_date FROM last_order_date) - INTERVAL '3 months'
GROUP BY 
    p.id, pn.name, c.name, pvs.stock, pvp.price, pv.weight, pv.status;
"""


In [4]:
# Load data from the database into a DataFrame
df = pd.read_sql(query, conn)

C:\Users\huawei\AppData\Local\Temp\ipykernel_14184\3069041871.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
# Data preprocessing (e.g., handle missing values, encoding, scaling)
df.fillna(0, inplace=True)


In [6]:
# Encode categorical columns (if necessary)
df = pd.get_dummies(df, drop_first=True)

In [7]:
# Features and target variable
X = df.drop(columns=['product_id', 'product_name', 'category_name'])
y = df['category_name']  # Assuming we are predicting the category (you can change this to a different target)


KeyError: "['product_id' 'product_name' 'category_name'] not found in axis"